In [6]:
import numpy as np
import pandas as pd
import subprocess
import argparse
from IPython.display import display

In [11]:
def mat(n_nodes=20):
    """
        This function computes the matrix from node number 

        Args :
        n_nodes : Number of nodes

        Returns:
        M : matrix (NxN) with N = 2 pow of the number of bits of n_nodes

    """
    bit_length = n_nodes.bit_length() # retrieving the number of bits needed to encode the number
    N = 2 ** bit_length # retrieving the size of the square matrix
    M = np.array([[np.nan]*N]*N) # initializing the matrix with nan of size 32*32
    #print(M.shape) # verification

    Na = N # address number

    for destination in range(Na):
        for source in range(Na):
            if source == destination or source ==0 or destination==0 or source == Na-1:
                continue
            if source > destination:
                M[destination][source] = destination*(Na-2) + (source-1)
            else:
                M[destination][source] = destination*(Na-2) + source
    
    return M

def get_probs(mat,network_mode=0):
    """
        This function computes the probability matrix

        Args :
        mat : matrix from number of nodes

        Returns:
        P : probability matrix of symbols
        P_symbols : one dimensional array containing each symbol's probability

    """
    # end getting symbols
    
    is_mixed = True if network_mode == 2 else False
    is_autonomous = True if network_mode==0 else False
    # start variable initialization
    P = np.zeros(mat.shape)
    N = len(mat)
    #P_symbols = np.zeros(2**N)
    Na = N
    Na2 = Na**2
    a_gate = 1
    p0 = (Na-2)**-1
    p1 = (Na-3)**-1
    p2 = (Na2 - 6*Na + 9)**-1
    # end variable initialization

    for destination in range(N):
        for source in range(N):
            gamma_a = 1
            gamma_c = 0 if is_autonomous else 1

            if is_mixed:
                gamma_a = 0.9
                gamma_c = 0.5
            if np.isnan(mat[destination,source]):
                P[destination,source] = np.nan
                continue
            if source==a_gate:
                p = (1-gamma_a)*p0
            elif destination == a_gate:
                p = (gamma_a-gamma_c)*p1
            elif source!=a_gate and destination!=a_gate:
                #gamma_a=gamma_c=1
                p = (gamma_c)*p2
            P[destination,source] = p
            #P_symbols[int(mat[destination,source])] = p
    return P#,P_symbols

#https://stackoverflow.com/questions/11587044/how-can-i-create-a-tree-for-huffman-encoding-and-decoding

def get_symbols(P):
    """
        Get symbols from P
    """
    # start getting symbols
    P[np.isnan(P)] = -1
    symbols = list(set(list(P.flatten())))
    symbols.remove(-1)
    return symbols

def get_occurence(M):
    """
        Get occurences from P
    """
    sym = get_symbols(M)
    T = []
    M = M.flatten()
    for i in sym:
        T.append(np.count_nonzero(M == i))
    return T,sym

def get_entropy(codes):
    probs = [i for i in codes.values()]
    entropy = sum([(np.log2(1/pi)*pi) for pi in probs])
    return entropy
def assign_code(nodes, label, result, prefix = ''):
    """
        assigning code to each nodes of the tree
    """    
    childs = nodes[label]     
    tree = {}
    if len(childs) == 2:
        tree['0'] = assign_code(nodes, childs[0], result, prefix+'0')
        tree['1'] = assign_code(nodes, childs[1], result, prefix+'1')     
        return tree
    else:
        result[label] = prefix
        return label

def Huffman_code(_vals): 
    """
        Computing the code with Huffman from the frequence probs

        Args:
            _vals : frequence matrix
        Returns:
            tree : the resulting tree 
            code : tree with code

    """   
    vals = _vals.copy()
    nodes = {}
    for n in vals.keys(): # leafs initialization
        nodes[n] = []

    while len(vals) > 1: # binary tree creation
        s_vals = sorted(vals.items(), key=lambda x:x[1]) 
        a1 = s_vals[0][0]
        a2 = s_vals[1][0]
        vals[a1+a2] = vals.pop(a1) + vals.pop(a2)
        nodes[a1+a2] = [a1, a2]        
    code = {}
    root = a1+a2
    tree = {}
    tree = assign_code(nodes, root, code)   # assignment of the code for the given binary tree      
    return code, tree

def decode_(encoded,tree):
    """
        Decode an encode input using the tree
    """
    decoded = []
    i = 0
    while i < len(encoded): # decoding using the binary graph
        ch = encoded[i]  
        act = tree[ch]
        while not isinstance(act, str):
            i += 1
            ch = encoded[i]  
            act = act[ch]        
        decoded.append(act)          
        i += 1
    return decoded

def encode_(plain,code):
    """
        Encode an input from a raw input
    """
    return ''.join([code[t] for t in plain])

def draw_tree(tree, prefix = ''): 
    """ 
        Draw tree from the computed  into a format undestandable by the grapviz dot
    """   
    if isinstance(tree, str):            
        descr = 'N%s [label="%s:%s", fontcolor=blue, fontsize=16, width=2, shape=box];\n'%(prefix, tree, prefix)
    else: # Node description
        descr = 'N%s [label="%s"];\n'%(prefix, prefix)
        for child in tree.keys():
            descr += draw_tree(tree[child], prefix = prefix+child)
            descr += 'N%s -> N%s;\n'%(prefix,prefix+child)
    return descr

def export_graph(tree,filename="graph.png"):

    """
        Export tree as a png image file from the formatted tree
    """

    with open('graph.dot','w') as f:
        f.write('digraph G {\n')
        f.write(draw_tree(tree))
        f.write('}') 
    subprocess.call('dot -Tpng graph.dot -o {}'.format(filename), shell=True)

def make_freq(P):
    """
        Make the frequence list of tuples (key: value)
    """
    P_tups = []
    for destination in range(len(P)):
        for source in range(len(P)):
            if np.isnan(P[destination,source]) or P[destination,source]==0:
                continue
            P_tups.append([P[destination,source], "{}-{}".format(destination,source)])
    return P_tups

def get_bits(raw):
    """
        Helper function to undo the tree formatting to get only bits (for countring puposes)
    """
    return raw.split(',')[0].split('=')[1].replace('"','').split(':')[1]

def sum_up_one(P):
    """
        Transform an array of probs to sum up to 1
    """
    P [np.isnan(P)] = 0.0
    Pt = []
    for id,i in enumerate(P):
        if sum(i)>0:
            Pt.append(i/sum(i))
        else:
            Pt.append(i)
    Pt = np.array(Pt).T
    Pt[Pt==0] = np.nan
    return Pt
def plot_mat(M,outpath=None,is_prob=False,network_mode=0):
    is_mixed = True if network_mode == 2 else False
    is_autonomous = True if network_mode==0 else False

    df = pd.DataFrame(M, columns=[str(i) for i in range(M.shape[1])])
    if outpath:
        if not is_mixed:
            df.to_csv(outpath+"/Matrix_{}_{}.csv".format("prob" if is_prob else "matrix","autonomous" if is_autonomous else "collaborative"))
        else:
            df.to_csv(outpath+"/Matrix_{}_{}.csv".format("prob" if is_prob else "matrix","mixed"))
    display(df)

def get_bin_codes(vals):
    bits = len(vals).bit_length()
    rets = vals.copy()
    for i,k in enumerate(vals.keys()):
        rets[k] = np.binary_repr(i, width=bits)    
    return rets



def main(n_nodes=6,network_mode=3,outpath="graph.png",verbose=False):
    """
        Executes the whole program from users input parameters
    """

    print("Computing with n_nodes : {} \n".format(n_nodes))

    M = mat(n_nodes)
    P = get_probs(M,network_mode)#[0]
    
    #P_old = P.copy()

    #P = sum_up_one(P)
    freq = make_freq(P)
    #print(freq)
    vals = {l:v for (v,l) in freq}
    #print(vals)
    codes, tree = Huffman_code(vals)
    #codes = get_bin_codes(vals) if len(list(set(vals.values())))==1 else code
    #if autonomous:

    #print(tree)

    t = draw_tree(tree).split("\n")
    #print(t) n_bits/(((2**(n_nodes.bit_length()))-1)**2)
    #tx = [ get_bits(i) for i in t if "fontcolor=blue" in i]
    #n_bits = np.sum(np.array([len(i) for i in tx]))
    n_bits =sum( [ len(k) for k in codes.values()])
    
    code_length = sum([ len(codes[k])*vals[k] for k in vals.keys()])


    if verbose:
        print("*"*20,"\n Matrix : \n ", "*"*20)
        plot_mat(M,outpath,is_prob=False,network_mode=network_mode)
        print("\n\n")

        print("*"*20,"\n Probabilities  : \n ", "*"*20)
        plot_mat(np.round_(P, decimals = 4),outpath,is_prob=True,network_mode=network_mode)
        print("\n\n")

        print("*"*20,"\n Huffman tree : \n ", "*"*20)
        print(tree)
        #print(t)
        print("\n\n")

        print("*"*20,"\n Code dictionary : \n ", "*"*20)
        print(codes)
        print("\n\n")

    if outpath:
        export_graph(tree,filename=outpath+"/graph.png")
    print("Leaf number : {}".format(len(t)))
    print("Huffman code : {}".format(n_bits))
    print("Entropy : {}".format(get_entropy(vals)))
    print("Huffman Code length : {} ".format(code_length))

#### Autonomous Network

In [31]:
main(8,network_mode=0,outpath="C:\\Users\\emman\\Documents\\resultats",verbose=True)

Computing with n_nodes : 8 

******************** 
 Matrix : 
  ********************


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,NaN
2,NaN,29.0,NaN,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,NaN
3,NaN,43.0,44.0,NaN,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,NaN
4,NaN,57.0,58.0,59.0,NaN,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,69.0,NaN
5,NaN,71.0,72.0,73.0,74.0,NaN,75.0,76.0,77.0,78.0,79.0,80.0,81.0,82.0,83.0,NaN
6,NaN,85.0,86.0,87.0,88.0,89.0,NaN,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,NaN
7,NaN,99.0,100.0,101.0,102.0,103.0,104.0,NaN,105.0,106.0,107.0,108.0,109.0,110.0,111.0,NaN
8,NaN,113.0,114.0,115.0,116.0,117.0,118.0,119.0,NaN,120.0,121.0,122.0,123.0,124.0,125.0,NaN
9,NaN,127.0,128.0,129.0,130.0,131.0,132.0,133.0,134.0,NaN,135.0,136.0,137.0,138.0,139.0,NaN





******************** 
 Probabilities  : 
  ********************


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.0769,0.0769,0.0769,0.0769,0.0769,0.0769,0.0769,0.0769,0.0769,0.0769,0.0769,0.0769,0.0769,NaN
2,NaN,0.0,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN
3,NaN,0.0,0.0000,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN
4,NaN,0.0,0.0000,0.0000,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN
5,NaN,0.0,0.0000,0.0000,0.0000,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN
6,NaN,0.0,0.0000,0.0000,0.0000,0.0000,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN
7,NaN,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN
8,NaN,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN
9,NaN,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,NaN





******************** 
 Huffman tree : 
  ********************
{'0': {'0': {'0': '1-12', '1': '1-13'}, '1': {'0': '1-14', '1': {'0': '1-2', '1': '1-3'}}}, '1': {'0': {'0': {'0': '1-4', '1': '1-5'}, '1': {'0': '1-6', '1': '1-7'}}, '1': {'0': {'0': '1-8', '1': '1-9'}, '1': {'0': '1-10', '1': '1-11'}}}}



******************** 
 Code dictionary : 
  ********************
{'1-12': '000', '1-13': '001', '1-14': '010', '1-2': '0110', '1-3': '0111', '1-4': '1000', '1-5': '1001', '1-6': '1010', '1-7': '1011', '1-8': '1100', '1-9': '1101', '1-10': '1110', '1-11': '1111'}



Leaf number : 50
Huffman code : 49
Entropy : 3.7004397181410926
Huffman Code length : 3.769230769230769 


### Collaborative Network

In [37]:
main(8,network_mode=1,outpath="C:\\Users\\emman\\Documents\\resultats",verbose=True)

Computing with n_nodes : 8 

******************** 
 Matrix : 
  ********************


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,NaN
2,NaN,29.0,NaN,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,NaN
3,NaN,43.0,44.0,NaN,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,NaN
4,NaN,57.0,58.0,59.0,NaN,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,69.0,NaN
5,NaN,71.0,72.0,73.0,74.0,NaN,75.0,76.0,77.0,78.0,79.0,80.0,81.0,82.0,83.0,NaN
6,NaN,85.0,86.0,87.0,88.0,89.0,NaN,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,NaN
7,NaN,99.0,100.0,101.0,102.0,103.0,104.0,NaN,105.0,106.0,107.0,108.0,109.0,110.0,111.0,NaN
8,NaN,113.0,114.0,115.0,116.0,117.0,118.0,119.0,NaN,120.0,121.0,122.0,123.0,124.0,125.0,NaN
9,NaN,127.0,128.0,129.0,130.0,131.0,132.0,133.0,134.0,NaN,135.0,136.0,137.0,138.0,139.0,NaN





******************** 
 Probabilities  : 
  ********************


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN
2,NaN,0.0,NaN,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,NaN
3,NaN,0.0,0.0059,NaN,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,NaN
4,NaN,0.0,0.0059,0.0059,NaN,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,NaN
5,NaN,0.0,0.0059,0.0059,0.0059,NaN,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,NaN
6,NaN,0.0,0.0059,0.0059,0.0059,0.0059,NaN,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,NaN
7,NaN,0.0,0.0059,0.0059,0.0059,0.0059,0.0059,NaN,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,NaN
8,NaN,0.0,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,NaN,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,NaN
9,NaN,0.0,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,0.0059,NaN,0.0059,0.0059,0.0059,0.0059,0.0059,NaN





******************** 
 Huffman tree : 
  ********************
{'0': {'0': {'0': {'0': {'0': {'0': {'0': '8-13', '1': '8-14'}, '1': {'0': '9-2', '1': '9-3'}}, '1': {'0': {'0': '9-4', '1': '9-5'}, '1': {'0': '9-6', '1': '9-7'}}}, '1': {'0': {'0': {'0': '9-8', '1': '9-10'}, '1': {'0': '9-11', '1': '9-12'}}, '1': {'0': {'0': '9-13', '1': '9-14'}, '1': {'0': '10-2', '1': '10-3'}}}}, '1': {'0': {'0': {'0': {'0': '10-4', '1': '10-5'}, '1': {'0': '10-6', '1': '10-7'}}, '1': {'0': {'0': '10-8', '1': '10-9'}, '1': {'0': '10-11', '1': '10-12'}}}, '1': {'0': {'0': {'0': '10-13', '1': '10-14'}, '1': {'0': '11-2', '1': '11-3'}}, '1': {'0': {'0': '11-4', '1': '11-5'}, '1': {'0': '11-6', '1': '11-7'}}}}}, '1': {'0': {'0': {'0': {'0': {'0': '11-8', '1': '11-9'}, '1': {'0': '11-10', '1': '11-12'}}, '1': {'0': {'0': '11-13', '1': '11-14'}, '1': {'0': '12-2', '1': '12-3'}}}, '1': {'0': {'0': {'0': '12-4', '1': '12-5'}, '1': {'0': '12-6', '1': '12-7'}}, '1': {'0': {'0': '12-8', '1': '12-9'}, '1': {'0': 

### Mixed network

In [36]:
main(8,network_mode=2,outpath="C:\\Users\\emman\\Documents\\resultats",verbose=True)

Computing with n_nodes : 8 

******************** 
 Matrix : 
  ********************


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,NaN
2,NaN,29.0,NaN,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,NaN
3,NaN,43.0,44.0,NaN,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,NaN
4,NaN,57.0,58.0,59.0,NaN,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,69.0,NaN
5,NaN,71.0,72.0,73.0,74.0,NaN,75.0,76.0,77.0,78.0,79.0,80.0,81.0,82.0,83.0,NaN
6,NaN,85.0,86.0,87.0,88.0,89.0,NaN,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,NaN
7,NaN,99.0,100.0,101.0,102.0,103.0,104.0,NaN,105.0,106.0,107.0,108.0,109.0,110.0,111.0,NaN
8,NaN,113.0,114.0,115.0,116.0,117.0,118.0,119.0,NaN,120.0,121.0,122.0,123.0,124.0,125.0,NaN
9,NaN,127.0,128.0,129.0,130.0,131.0,132.0,133.0,134.0,NaN,135.0,136.0,137.0,138.0,139.0,NaN





******************** 
 Probabilities  : 
  ********************


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.0308,0.0308,0.0308,0.0308,0.0308,0.0308,0.0308,0.0308,0.0308,0.0308,0.0308,0.0308,0.0308,NaN
2,NaN,0.0071,NaN,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,NaN
3,NaN,0.0071,0.0030,NaN,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,NaN
4,NaN,0.0071,0.0030,0.0030,NaN,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,NaN
5,NaN,0.0071,0.0030,0.0030,0.0030,NaN,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,NaN
6,NaN,0.0071,0.0030,0.0030,0.0030,0.0030,NaN,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,NaN
7,NaN,0.0071,0.0030,0.0030,0.0030,0.0030,0.0030,NaN,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,NaN
8,NaN,0.0071,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,NaN,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,NaN
9,NaN,0.0071,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,NaN,0.0030,0.0030,0.0030,0.0030,0.0030,NaN





******************** 
 Huffman tree : 
  ********************
{'0': {'0': {'0': {'0': {'0': {'0': {'0': {'0': '9-8', '1': '9-10'}, '1': {'0': '9-11', '1': '9-12'}}, '1': {'0': {'0': '9-13', '1': '9-14'}, '1': {'0': '10-2', '1': '10-3'}}}, '1': {'0': {'0': {'0': '10-4', '1': '10-5'}, '1': {'0': '10-6', '1': '10-7'}}, '1': {'0': {'0': '10-8', '1': '10-9'}, '1': {'0': '10-11', '1': '10-12'}}}}, '1': {'0': {'0': {'0': {'0': '10-13', '1': '10-14'}, '1': {'0': '11-2', '1': '11-3'}}, '1': {'0': {'0': '11-4', '1': '11-5'}, '1': {'0': '11-6', '1': '11-7'}}}, '1': {'0': {'0': {'0': '11-8', '1': '11-9'}, '1': {'0': '11-10', '1': '11-12'}}, '1': {'0': {'0': '11-13', '1': '11-14'}, '1': {'0': '12-2', '1': '12-3'}}}}}, '1': {'0': {'0': {'0': {'0': {'0': '12-4', '1': '12-5'}, '1': {'0': '12-6', '1': '12-7'}}, '1': {'0': {'0': '12-8', '1': '12-9'}, '1': {'0': '12-10', '1': '12-11'}}}, '1': {'0': {'0': {'0': '12-13', '1': '12-14'}, '1': {'0': '13-2', '1': '13-3'}}, '1': {'0': {'0': '13-4', '1': '13-